# Intro to Machine Learning with scikit-learn

## About me - Ted Petrou

* Author 
    * Pandas Cookbook
    * [Exercise Python][1]
    * [Master Data Analysis with Python][2]
    * [Master Machine Learning with Python][3]
* Founder of [Dunder Data][4]
* Author of [Dexplo and Dexplot][5]


[1]: https://www.dunderdata.com/exercise-python
[2]: https://www.dunderdata.com/master-data-analysis-with-python
[3]: https://www.dunderdata.com/master-machine-learning-with-python
[4]: https://dunderdata.com
[5]: https://github.com/dexplo

## Topics

* What is machine learning?
* Exploratory data analysis
* Modeling by hand
* The scikit-learn Estimator
* Different regression estimators
* Cross validation
* Grid search

## Learning vs Machine Learning

### What is Learning?
Learning is the ability to improve at a **task**. Learning is done by animals, humans, and some machines.

### What is a task?
A task is a clearly defined piece of work.

### Measuring task performance
Learning happens when the person or machine improves its performance at completing the task. 

### What is Machine Learning?
Machine learning is often defined as the ability of a machine to learn (to improve on a specific task) without being explicitly programmed to do so.

### What is "not explicitly programmed"?
Not updated by a human

### The two types of machine learning
* **supervised** - have labels (ground truth)
* **unsupervised** - no labels

### Regression vs Classification
* **regression** - continuous value labels
* **classification** - discrete labels


## Terminology

![][1]


## Assessing task performance
Objectively quantifiable measure of performance

### Assessing regression task performance
Minimize error

[1]: images/terminology.png

## Ames Housing Data

* [Famous beginners Kaggle competition][0] compiled by professor Dean De Cock from Ames, Iowa from 2006 - 2010
* Original dataset has 79 features and 1460 samples
* For simplicity, we will only look at 8 features
* Predict sale price
* Evaluation metric - minimize squared error

[0]: https://www.kaggle.com/c/house-prices-advanced-regression-techniques

### Read in Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
housing = pd.read_csv('data/housing_sample.csv')
housing.head()

### Data dictionary

Open up the [data dictionary for these columns][0] in a separate tab and read through the column descriptions.

[0]: data/housing_sample_data_dictionary.txt

## Exploratory data analysis

Gain an understanding of the data without the use of formal modeling.

* Relies heavily on visualization
* Begin with univariate analysis
    * Categorical features (limited, known, discrete values)
        * Frequency of occurrence of each value
    * Continuous features ()
        * Summary statistics (min, max, mean, median, std, etc...)
        * Distribution plots (boxplot, histogram, KDE)
* Relationship with target
    * Compare each feature with target
    * Categorical features
        * Mean sale price by category
    * Continuous features
        * Scatterplot of feature vs sale price
        * Bin feature and take mean per bin
        * Correlation vs sale price
* Multivariate analysis
    * All categorical features - crosstabulation, pivot table with sales price
    * All continuous - scatterplots
    * Mix of categorical and continuous - distribution plot by group

### Basic information on data

* Number of features and observations
* Data type of each column
* Number of missing values per column

In [ ]:
housing.shape

In [ ]:
housing.dtypes

In [ ]:
housing.isna().sum()

### Univariate analysis

Begin with categorical columns

* Neighborhood
* Exterior1st
* BedroomAbvGr
* FullBath

In [ ]:
categorical_cols = ['Neighborhood', 'Exterior1st', 'BedroomAbvGr', 'FullBath']

In [ ]:
housing['Neighborhood'].value_counts().plot(kind='bar', figsize=(10, 4));

In [ ]:
housing['Exterior1st'].value_counts().plot(kind='bar', figsize=(10, 4));

In [ ]:
housing['BedroomAbvGr'].value_counts().sort_index().plot(kind='bar', figsize=(10, 4));

In [ ]:
housing['FullBath'].value_counts().sort_index().plot(kind='bar', figsize=(10, 4));

### Continuous columns


* YearBuilt	
* LotFrontage	
* GrLivArea	
* GarageArea

In [ ]:
continuous_cols = ['YearBuilt', 'LotFrontage', 'GrLivArea', 'GarageArea']
fig, axes = plt.subplots(4, 2, figsize=(10, 12))
for i, col in enumerate(continuous_cols):
    housing[col].plot(kind='box', vert=False, ax=axes[i, 0]);
    axes[i, 1] = housing[col].plot(kind='hist', ax=axes[i, 1], title=col);
fig.tight_layout()

### Relationship with target

Categorical columns

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8), sharey=True)
for col, ax in zip(categorical_cols, axes.flatten()):
    housing.groupby(col)['SalePrice'].mean().plot(kind='bar', ax=ax)
fig.tight_layout()

Continuous features

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8), sharey=True)
for col, ax in zip(continuous_cols, axes.flatten()):
    housing.plot(x=col, y = 'SalePrice', kind='scatter', ax=ax)
fig.tight_layout()

### Multivariate Analysis

In [ ]:
pd.crosstab(index=housing['Neighborhood'], columns=housing['FullBath'])

Average sale price for bedroom and bathroom combination.

In [ ]:
housing.pivot_table(index='BedroomAbvGr', columns='FullBath', 
                    values='SalePrice', aggfunc='mean').round(-3)

Use seaborn to plot regression line of above ground living area vs sale price. Make a separate line for each unique bedroom.

In [ ]:
sns.lmplot(x='GrLivArea', y='SalePrice', data=housing, hue='BedroomAbvGr', aspect=2, ci=None) \
   .set(xlim=(500, 3000), ylim=(0, 500000));

In [ ]:
gr_bins = pd.cut(housing['GrLivArea'], range(500, 4000, 500))
housing.pivot_table(index='Neighborhood', columns=gr_bins, 
                    values='SalePrice', aggfunc='mean') // 1000

#### Correlation

In [ ]:
housing.corrwith(housing['SalePrice']).round(2).sort_values(ascending=False)

## Build a simple model without scikit-learn

scikit-learn automates the learning process for you. Instead of beginning with it, I recommend building a simple model without it.

### What are we predicting?
In this problem, we want to predict the final sale price of the house.

### Assign target variable to `y`

In [ ]:
y = housing['SalePrice']
y.head()

### Baseline model for regression

One of the simplest models we can build for regression problems is guessing the mean.

In [ ]:
y_pred = y.mean()
y_pred

### Assess model performance

For each model we build, we will assess its performance. A popular measure of performance for regression problems is the Sum of Squared Error or SSE. This calculation involves three steps:

* Calculate error for each observation - actual - mean
* Square each error
* Sum the errors

#### Step 1: Calculate error

In [ ]:
error = y - y_pred
error.head()

#### Step 2: Square errors

In [ ]:
error_squared = error ** 2
error_squared.head()

#### Step 3: Sum errors

Get a single metric to objectively quantify our model.

In [ ]:
model_1_sse = error_squared.sum()
model_1_sse

### Model 2 - Use average neighborhood sale price

In [ ]:
model_2 = housing.groupby('Neighborhood')['SalePrice'].mean()
model_2.head()

In [ ]:
y_pred = model_2.loc[housing['Neighborhood']].values.round(-3)
y_pred[:10]

#### Calculate SSE in one step

In [ ]:
model_2_sse = ((y - y_pred) ** 2).sum()
model_2_sse

### Model 3

In [ ]:
housing['GrLivArea'].max()

In [ ]:
gr_bins = pd.cut(housing['GrLivArea'], range(0, 6500, 500))
model_3 = housing.groupby(['Neighborhood', gr_bins])['SalePrice'].mean()
model_3.head(20)

In [ ]:
X = housing[['Neighborhood', 'GrLivArea']].itertuples(index=False)
y_pred = model_3[X].values.round(-3)
y_pred[:5]

In [ ]:
model_3_sse = ((y - y_pred) ** 2).sum()
model_3_sse

### Plot errors for simple models

In [ ]:
errors = {'model_1_simple_avg': model_1_sse,
          'model_2_neigh_avg': model_2_sse, 
          'model_3_neigh_area_avg': model_3_sse}
pd.Series(errors).plot(kind='bar', figsize=(12, 5), rot=0);

## Machine Learning with scikit-learn

* scikit-learn provides many regression models to learn from data
* The goal of each model is to minimize the squared error 

### Linear Regression

Linear regression is one of the simpler models available in scikit-learn. When learning with a single feature, it's called **simple linear regression**. The form of the model is as follows:

$$\hat{y} = w_{0} + w_{1}x_{1}$$

Where $\hat{y}$ (pronounced y-hat) is the predicted value, $x_1$ is the feature value, and $w_0$ and $w_1$ are the parameters. This is the equation or a line where $w_0$ is the y-intercept and $w_1$ is the slope. 

### Goal of linear regression

The goal of all models is to learn from data. With linear regression, we would like to find the value of the parameters ($w_0$ and $w_1$) that minimize the SSE. Once the values for the parameters are found then the learning is complete.

### Select a single feature to learn from

We'll select above ground living area (`GrLivArea`) as our single feature for our first linear regression model. Let's plot the relationship between it and the sale price as a scatterplot.

In [ ]:
housing.plot(x='GrLivArea', y='SalePrice', kind='scatter', figsize=(12, 6));

### Choose values of parameters by hand

Before using scikit-learn, let's choose a couple different combinations of $w_0$ and $w_1$ and plot the lines and calculate error. First we create a function that makes the predictions for us given the parameters and the feature values.

In [ ]:
def lr_predict(w0, w1, x):
    return w0 + w1 * x

Let's use this function to plot two models with different parameter values.

In [ ]:
ax = housing.plot(x='GrLivArea', y='SalePrice', kind='scatter', figsize=(12, 6));

# model 1 intercept of 50,000 and slope of 75
x = np.array([[0], [5000]])
w0, w1 = 50_000, 75
ax.plot(x, lr_predict(w0, w1, x), color='red', lw=3, label=f'$w_0={w0:,}$\n$w_1={w1}$')

# model 2 intercept of 0 and slope of 125
w0, w1 = 0, 125
ax.plot(x, lr_predict(w0, w1, x), color='green', lw=3, label=f'$w_0={w0}$\n$w_1={w1}$')
ax.legend();

### Calculate errors for each model

Use root mean squared error (RMSE) instead. This is a very similar metric, but is relative and not absolute. Much smaller and easier to compare.

In [ ]:
y_pred = lr_predict(50_000, 75, housing['GrLivArea'])
y_pred.head()

In [ ]:
model_1_lr = np.sqrt(((y - y_pred) ** 2).mean()).round()
model_1_lr

In [ ]:
y_pred = lr_predict(0, 125, housing['GrLivArea'])
model_2_lr = np.sqrt(((y - y_pred) ** 2).mean()).round()
model_2_lr

Model two has lowest error and is best.

## The scikit-learn Estimator

We now turn to scikit-learn to automate the learning process. scikit-learn will quickly find us the best combination of linear regression parameter values that minimize the squared error.

scikit-learn uses the term **Estimator** to refer to any object that learns from data. There are several types of estimators that scikit-learn has built for us. Not all estimators are machine learning models, but all of them **learn from data**. Here are the main types of estimators:

* Regressors - Supervised learning with continuous target
* Classifiers - Supervised learning with categorical target
* Clusterers - Unsupervised learning
* Transformers - Transform the input/output data
* Meta-estimators - Learn from other estimators

### Understanding the scikit-learn API

It is instructive to take a glimpse of the scikit-learn API before using it. I like to analogize the API to a house where each of the rooms contains the useful objects for machine learning. Take a look at the scikit-learn 'house' below:

![][1]

[1]: images/scikit_house.png

## Linear regression in scikit-learn

While the above models look reasonable, they do not minimize the squared error. scikit-learn uses an algorithm that guarantees to find the combination of parameters that minimize the SSE.

### Data requirements for scikit-learn

In order to use scikit-learn, the data must be provided in a particular form. Here is how you need to provide your data:

* The input data must be two-dimensional
* The output data can be one-dimensional
* There can be no missing values
* The input data cannot contain strings

### Select feature as a DataFrame and Target as a Series

scikit-learn requires our input data to be two dimensional, even if we are using just a single feature as our input. Below, we select our feature as a DataFrame using a single-item list. By convention, scikit-learn uses the variable names `X` and `y`.

In [ ]:
X = housing[['GrLivArea']]
y = housing['SalePrice']
X.head()

## Three-step process for all Estimators - Import, Instantiate, Fit

All scikit-learn estimators follow the same three step process:

* Import - find model in scikit-learn 'house'
* Instantiate - create a single instance of the model
* Fit - learn from data

### Step 1: Import

### Step 2: Instantiate

### Step 3: Fit

### Get the model

After calling the `fit` method, the model is trained. We can find the optimal parameter values by accessing the `intercept_` and `coef_` attributes.

### Make predictions with the `predict` method

### Make predictions on all input data

### Plot best model

In [ ]:
ax = housing.plot(x='GrLivArea', y='SalePrice', kind='scatter', figsize=(12, 6))

# model 1 intercept of 50,000 and slope of 75
x = np.array([[0], [5000]])
w0, w1 = 50_000, 75
ax.plot(x, lr_predict(w0, w1, x), color='red', lw=3, label=f'$w_0={w0:,}$\n$w_1={w1}$')

# model 2 intercept of 0 and slope of 125
w0, w1 = 0, 125
ax.plot(x, lr_predict(w0, w1, x), color='green', lw=3, label=f'$w_0={w0}$\n$w_1={w1}$')

# LinearRegression in scikit-learn
y_pred = lr.predict(x)
w0 = lr.intercept_.round(-3)
w1 = lr.coef_[0]
ax.plot(x, y_pred, color='black', lw=5, ls='--', label=f'$w_0={w0:,.0f}$\n$w_1={w1:.0f}$')
ax.legend();

### Assessing model performance - $R^2$

scikit-learn uses, $R^2$, a slightly different metric as its default for model performance. Here is the procedure for calculating $R^2$

* Find the SSE of the model
* Find the SSE when guessing the mean
* $R^2$ is the percentage decrease in the SSE when guessing the mean versus the SSE of the model.

### Calculate $R^2$ by hand

In [ ]:
y_pred_model = lr.predict(X)
y_pred_mean = y.mean()

In [ ]:
sse_model = ((y - y_pred_model) ** 2).sum()
sse_model

In [ ]:
sse_mean = ((y - y_pred_mean) ** 2).sum()
sse_mean

In [ ]:
r2 = 1 - sse_model / sse_mean
r2

### Use score method to calculate $R^2$

All regression estimators have a `score` method that returns the $R^2$. Let's verify that it's the same.

### Visualizing $R^2$

Best possible score is 1.

![][0]

[0]: images/r2_matplotlib.png

### $R^2$ - percent variance explained

The SSE when guessing the mean is also the definition of the statistical variance (multiplied by the number of observations). Therefore, the SSE of the model is the percentage decrease in this variance. The variance is the natural variation of the target. We are interested in understanding why it varies. Why isn't the target the same for every input? We use a model to explain this variance. We'd like to explain all of the variance. The metric $R^2$ informs us of how much this natural inherent variance has been removed.

## Exercise

Instantiate a new `LinearRegression` estimator and use a different feature to learn from. Output the parameter values and the $R^2$ score.

## Multiple Linear Regression

Use more than one predictor variable. New model:

$$\hat{y} = w_{0} + w_{1}x_{1} + w_{2}x_{2} + ... + w_{p}x_{p}$$

Where `p` is the number of predictor variables (features).

### Select multiple columns

In [ ]:
cols = ['GrLivArea', 'GarageArea', 'FullBath']
X = housing[cols]
X.head()

### Train the model again - no need to import

Since we already imported the `LinearRegression` class, we can straight to step 2.

Fit with new data.

### Get learned model

### Model interpretation

For every one unit increase in the feature, a corresponding increase of the parameter value is expected in the target (many assumptions).

### Score new model

## Other ML models - Back at the scikit-learn house

![][0]

[0]: images/scikit_house.png

## K-Nearest Neighbors

K-Nearest neighbors (KNN) provides a completely different approach to learning from data. KNN finds the most similar observations to the one being predicted. It then averages the target variable for all of these similar neighbors to make a prediction. You must supply it a value, `k`, for the number of neighbors you would like to find.

### Visualizing KNN

The below function accepts a value for an above ground living area, garage area, and number of neighbors. It then calculates the distance between the provided values and all other values in the dataset. The k nearest neighbors are found based on this distance calculation. The average sales price of these five neighbors is returned.

In [ ]:
def plot_neighbors(sq_foot, sq_foot_garage, k):
    df = housing[['GrLivArea', 'GarageArea', 'SalePrice']].copy()
    df['distance'] = np.sqrt((df['GrLivArea'] - sq_foot) ** 2 + (df['GarageArea'] - sq_foot_garage) ** 2)
    df_neighbors = df.nsmallest(k, 'distance')
    mean = df_neighbors['SalePrice'].mean()
    ax = df.plot(x='GrLivArea', y='GarageArea', kind='scatter', figsize=(14, 6))
    df_neighbors.plot(x='GrLivArea', y='GarageArea', kind='scatter', color='red', ax=ax, label=f'{k} neighbors')
    ax.scatter([sq_foot], [sq_foot_garage], color='green', s=40, label='Predicted Value')
    ax.set_title(f'{k} nearest neighbors of {sq_foot} square feet and {sq_foot_garage} '
                 f'basement square feet predict sale price of {mean}')
    ax.legend()

In [ ]:
plot_neighbors(3500, 700, 5)

### KNN in scikit-learn

The `KNeighborsRegressor` estimator from the `neighbors` module is available in scikit-learn to do KNN. Just like all estimators, it follows the same three-step process - import, instantiate, fit. When we instantiate it, we set the number of neighbors with the `n_neighbors` parameter. All three steps are completed in a single cell. We select the same features as the plot above.

In [ ]:
cols = ['GrLivArea', 'GarageArea']
X = housing[cols]

### Prediction

### Score

By default, all scikit-learn regression estimators use $R^2$ as their evaluation metric in the `score` method.

## Decision Trees

Decision trees are one of the more interpretable models (when not so deep) and are composed of:

* Nodes - where a binary decision is made
* Branches - lead you to the next node (decision)
* Leaves - bottom of the tree, where prediction is

![][0]



[0]: images/tree.png

### Decision trees in scikit-learn

The `DecisionTreeRegressor` estimator from the `tree` module learns a decision tree in scikit-learn. Let's use a different set of columns.

In [ ]:
cols = ['BedroomAbvGr', 'FullBath']
X = housing[cols]
X.head()

Follow the same three step process. Set the maximum depth of the tree with the `max_depth` parameter.

### Make predictions and score

### Visualizing the decision tree

You'll need to install the graphviz library with `conda install graphviz`.

In [ ]:
import graphviz
from sklearn.tree import export_graphviz
dot_data = export_graphviz(dtr, out_file=None, feature_names=cols, precision=1,
                           filled=True, rounded=True,  special_characters=True)  
graph = graphviz.Source(dot_data, format='png')  
graph

### Get nearly a perfect score

Decision trees are very flexible models and can provide a nearly perfect fit to the data it is trained on. Let's build a new decision on a different set of features.

In [ ]:
cols = ['GrLivArea', 'GarageArea', 'FullBath']
X = housing[cols]
X.head()

If we do not set the `max_depth` parameter, then there will be no limit to how deep the tree can grow. It will grow until only a single observation remains in each node. With no limit to it's depth, we get nearly a perfect score.

## Model evaluation

Thus far, we've evaluated ourselves on the data that the model was trained on. Instead, we must evaluate ourselves on unseen data. One procedure for proper evaluation is cross validation.

* Divide data into `k` equal partitions
* Train a model on `k - 1` partitions
* Evaluate the model on the 1 partition not used during training
* Record the score on that 1 partition
* Repeat process using a different partition for evaluation
* Continue until all partitions have been used for evaluation

Many different flavors of cross validation. This procedure is called K-fold cross validation where the 'fold' is a partition of the data and 'k' is the number of these partitions. This value is usually between 5 and 10.

![][0]

[0]: images/kfold.png

### Cross validation in scikit-learn

The `cross_val_score` function automates the entire procedure for us. Pass it the instantiated estimator, input and output data and set, and the number of folds with the `cv` parameter.

In [ ]:
cols = ['GrLivArea', 'GarageArea', 'FullBath']
X = housing[cols]
X.head()

### Cross validation approximates our future performance

We want to know how well our model performs in the future. Cross validation is a procedure for approximating this future performance.

## Tune Hyperparameters with grid-search

All models have hyperparameters that change the behavior of the model. These are set during instantiation in step 2. Hyperparameters are NOT learned during training. They are set by you. You choose what they are. Depending on the model, these hyperparameters can drastically change the performance of the model. Decision trees are heavily influenced by their hyperparameter values. 

### Using the `GridSearchCV` metaestimator

Instead of manually iterating over different combinations of hyperparameters, scikit-learn provides the meta-estimator `GridSearchCV` to automate the process for us. To use it, you must create a grid using a dictionary mapping the hyperparameter name as a string to the possible values (as a list or array).

The same three-step process exists for meta-estimators. We import it and instantiate it by passing it the instantiated model, the grid, and the number of folds.

### Grid search results

See all results at once:

In [ ]:
df_results = pd.DataFrame(gs.cv_results_)
df_results

In [ ]:
df_results.pivot(index='param_max_depth', columns='param_min_samples_leaf', 
                 values='mean_test_score').round(3).style.background_gradient('coolwarm', axis=None)